In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as pltimport

In [3]:
train=pd.read_csv("train.csv")
test=pd.read_csv("test.csv")
print(train.shape)
print(test.shape)

(1460, 81)
(1459, 80)


In [4]:
combined=pd.concat([train,test])
combined.shape #2919
#combined.iloc[:1460:,:] #train
#combined.iloc[1460:,:] #test

C:\Users\samuelmao\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  """Entry point for launching an IPython kernel.


(2919, 81)

In [5]:
combined=combined.reset_index()

In [6]:
from preprocess import impute
all_data, encodedDic = impute (combined, False)
all_data_onehot, encodedDic = impute (combined, True) #one-hot

In [7]:
train=all_data.iloc[:1460:,:] #train
train_y=train["SalePrice"]
train_x=train[train.columns[train.columns!="SalePrice"]]
test=test.iloc[1460:,:]

train_onehot=all_data_onehot.iloc[:1460:,:] #train
train_y_onehot=train_onehot["SalePrice"]
train_x_onehot=train_onehot[train_onehot.columns[train_onehot.columns!="SalePrice"]]
test_onehot=train_onehot.iloc[1460:,:]


## RF: label encoding

In [42]:
##Grid Search
from sklearn import ensemble
from sklearn import model_selection
randomForest = ensemble.RandomForestRegressor()
grid_para_forest = [{
    "n_estimators": [120],
    "max_features": ["sqrt"],
    "criterion": ["mse"],
    "min_samples_leaf": [1],
    "min_samples_split": [4],
    "oob_score": [True],
    "random_state": [42]}]
grid_search_forest = model_selection.GridSearchCV(randomForest, grid_para_forest, scoring='neg_mean_squared_error', cv=5, n_jobs=-1)
grid_search_forest.fit(train_x, train_y)

GridSearchCV(cv=5, error_score='raise',
       estimator=RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
           oob_score=False, random_state=None, verbose=0, warm_start=False),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid=[{'n_estimators': [120], 'max_features': ['sqrt'], 'criterion': ['mse'], 'min_samples_leaf': [1], 'min_samples_split': [4], 'oob_score': [True], 'random_state': [42]}],
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='neg_mean_squared_error', verbose=0)

In [25]:
print(grid_search_forest.best_params_)
print(grid_search_forest.best_score_)

{'criterion': 'mse', 'max_features': 'sqrt', 'min_samples_leaf': 1, 'min_samples_split': 4, 'n_estimators': 120, 'oob_score': True, 'random_state': 42}
-848421620.3827785


In [57]:
## Model fitting
from sklearn.model_selection import train_test_split
from sklearn import ensemble
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt

X_train, X_test, y_train, y_test = train_test_split(train_x, train_y, test_size=0.3, random_state=98)
params={"n_estimators":120, "max_features": "sqrt", "criterion": "mse", "min_samples_leaf": 1, "min_samples_split": 4,"oob_score": True,"random_state": 42}
randomForest = ensemble.RandomForestRegressor(**params)
randomForest.fit(X_train,y_train)
mse=mean_squared_error(y_test, randomForest.predict(X_test))
mse


931896202.7753068

In [ ]:
feature_importance = list(zip(train_x.columns[:-2], randomForest.feature_importances_))
dtype = [('feature', 'S10'), ('importance', 'float')]
feature_importance = np.array(feature_importance, dtype=dtype)
feature_sort = np.sort(feature_importance, order='importance')[::-1]
feature_sort

## RF one-hot

In [58]:
##Grid Search
from sklearn import ensemble
from sklearn import model_selection
randomForest = ensemble.RandomForestRegressor()
grid_para_forest = [{
    "n_estimators": [120],
    "max_features": ["sqrt"],
    "criterion": ["mse"],
    "min_samples_leaf": [1],
    "min_samples_split": [4],
    "oob_score": [True],
    "random_state": [42]}]
grid_search_forest = model_selection.GridSearchCV(randomForest, grid_para_forest, scoring='neg_mean_squared_error', cv=5, n_jobs=-1)
grid_search_forest.fit(train_x_onehot, train_y_onehot)

GridSearchCV(cv=5, error_score='raise',
       estimator=RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
           oob_score=False, random_state=None, verbose=0, warm_start=False),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid=[{'n_estimators': [120], 'max_features': ['sqrt'], 'criterion': ['mse'], 'min_samples_leaf': [1], 'min_samples_split': [4], 'oob_score': [True], 'random_state': [42]}],
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='neg_mean_squared_error', verbose=0)

In [59]:
print(grid_search_forest.best_params_)
print(grid_search_forest.best_score_)

{'criterion': 'mse', 'max_features': 'sqrt', 'min_samples_leaf': 1, 'min_samples_split': 4, 'n_estimators': 120, 'oob_score': True, 'random_state': 42}
-912016494.9534297


In [60]:
## Model fitting
from sklearn.model_selection import train_test_split
from sklearn import ensemble
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt

X_train, X_test, y_train, y_test = train_test_split(train_x_onehot, train_y_onehot, test_size=0.3, random_state=98)
params={"n_estimators":120, "max_features": "sqrt", "criterion": "mse", "min_samples_leaf": 1, "min_samples_split": 4,"oob_score": True,"random_state": 42}
randomForest = ensemble.RandomForestRegressor(**params)
randomForest.fit(X_train,y_train)
mse=mean_squared_error(y_test, randomForest.predict(X_test))
mse


886540227.1488433

In [62]:
feature_importance = list(zip(train_x_onehot.columns, randomForest.feature_importances_))
dtype = [('feature', 'S10'), ('importance', 'float')]
feature_importance = np.array(feature_importance, dtype=dtype)
feature_sort = np.sort(feature_importance, order='importance')[::-1]
feature_sort

array([(b'OverallQua', 8.93884629e-02), (b'TotalFlrSF', 7.60985762e-02),
       (b'GrLivArea', 6.64768126e-02), (b'YearBuilt', 5.08041362e-02),
       (b'GarageCars', 4.61653310e-02), (b'TotalBsmtS', 3.82928249e-02),
       (b'BsmtQual', 3.77060766e-02), (b'KitchenQua', 3.33367002e-02),
       (b'GarageArea', 3.29475059e-02), (b'1stFlrSF', 3.26868442e-02),
       (b'FullBath', 2.75943618e-02), (b'Bsmt_GLQ', 2.63198705e-02),
       (b'GarageYrBl', 2.48776676e-02), (b'YearRemodA', 2.24358899e-02),
       (b'TotRmsAbvG', 2.23495488e-02), (b'ExterQual', 2.19681485e-02),
       (b'LotArea', 2.14525436e-02), (b'FireplaceQ', 2.03310588e-02),
       (b'2ndFlrSF', 1.84905930e-02), (b'Fireplaces', 1.61424316e-02),
       (b'MasVnr_Brk', 1.48874618e-02), (b'OpenPorchS', 1.35584485e-02),
       (b'Foundation', 1.17628528e-02), (b'LotFrontag', 1.10707430e-02),
       (b'HeatingQC', 7.54744374e-03), (b'WoodDeckSF', 7.50954472e-03),
       (b'TotalProch', 7.03355229e-03), (b'MSSubClass', 6.83015880e-